In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)

CUDA available: True


In [34]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation

        self.fc_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.fc_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.output_layer = nn.Linear(prev_size, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        x = self.output_layer(x)
        return x

In [35]:
from sklearn.datasets import make_classification

# Generate a classification dataset
X, y = make_classification(n_samples=500, n_features=20, n_informative=10, n_classes=2)

# Print the shape of the dataset
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (500, 20)
Shape of y: (500,)


In [41]:
from sklearn.model_selection import train_test_split
import numpy as np
import time

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.025, random_state=42)

# Create data loaders for the training and testing sets
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the model architecture and initialize the model instance
input_size = X.shape[1]
hidden_sizes = [32, 32, 16]
output_size = 2
activation = nn.ReLU()
model = MLP(input_size, hidden_sizes, output_size, activation)

# Define the optimizer
criterion = nn.CrossEntropyLoss()

# Define the loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass the input through the model
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels.long())

        # Backpropagate the gradients
        loss.backward()

        # Update the model parameters
        optimizer.step()

# Evaluate the model on the testing set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

    accuracy = correct / total
    print("Accuracy:", accuracy)


Using GPU
Accuracy: 0.9230769230769231
